**１.driveマウント**

In [1]:
# @title
# === 1) Google Drive マウント ===
from google.colab import drive
drive.mount("/content/drive")

# === 2) 作業ディレクトリ設定 ===
import os
BASE_DIR = "/content/drive/MyDrive/Inventory_monitoring"
os.chdir(BASE_DIR)

print("現在の作業ディレクトリ:", os.getcwd())
print("ディレクトリ構成:")

# サブフォルダの内容を一覧表示
for folder in ["sql", "data", "util"]:
    path = os.path.join(BASE_DIR, folder)
    if os.path.exists(path):
        files = os.listdir(path)
        print(f"  ┣ {folder}/ ({len(files)} files)")
        for f in files[:10]:  # 上位10件まで表示（多い場合は省略）
            print(f"     ┗ {f}")
        if len(files) > 10:
            print("     ...")
    else:
        print(f"  ┣ {folder}/ (not found)")

# === 3) Google アカウント認証 ===
from google.colab import auth
auth.authenticate_user()
print("Google 認証 OK")

# === 4) BigQuery クライアント初期化 ===
from google.cloud import bigquery
PROJECT_ID = "tential-data-prd"
bq = bigquery.Client(project=PROJECT_ID)
print(f"BigQuery 接続 OK: {bq.project}")


Mounted at /content/drive
現在の作業ディレクトリ: /content/drive/MyDrive/Inventory_monitoring
ディレクトリ構成:
  ┣ sql/ (6 files)
     ┗ 1.sales.sql
     ┗ 3.purchase.sql
     ┗ 4,purchase_plan.sql
     ┗ 5.invent_eom.sql
     ┗ 6.transfer.sql
     ┗ 2.revenue.sql
  ┣ data/ (1 files)
     ┗ 仕入予定(未登録分).gsheet
  ┣ util/ (6 files)
     ┗ common.py
     ┗ init.py
     ┗ __pycache__
     ┗ bq_io.py
     ┗ forecasting.py
     ┗ psi.py
Google 認証 OK
BigQuery 接続 OK: tential-data-prd


**2.データフレーム格納**

In [2]:
# @title
# === 必要モジュール ===
from pathlib import Path
from datetime import date
from util.bq_io import run_sql_file_to_df  # BigQuery 実行関数

# === SQL ファイル定義 ===
SQL_FILES = {
    "sales":         Path("sql/1.sales.sql"),
    "revenue":       Path("sql/2.revenue.sql"),
    "purchase":      Path("sql/3.purchase.sql"),
    "purchase_plan": Path("sql/4,purchase_plan.sql"),
    "invent_eom":    Path("sql/5.invent_eom.sql"),
    "transfer":      Path("sql/6.transfer.sql"),
}

# === 共通クエリパラメータ ===
QUERY_PARAMS = {
    "today": date.today().isoformat(),  # 例: '2025-10-28'
}

# === 実行＆確認 ===
frames = {}
for key, path in SQL_FILES.items():
    df = run_sql_file_to_df(path, QUERY_PARAMS)
    frames[key] = df
    print(f"[OK] {key:14s} <- {path.name:30s} shape={df.shape}")


[OK] sales          <- 1.sales.sql                    shape=(14605, 7)
[OK] revenue        <- 2.revenue.sql                  shape=(3609, 4)
[OK] purchase       <- 3.purchase.sql                 shape=(2607, 4)
[OK] purchase_plan  <- 4,purchase_plan.sql            shape=(2001, 4)
[OK] invent_eom     <- 5.invent_eom.sql               shape=(10386, 4)
[OK] transfer       <- 6.transfer.sql                 shape=(937, 5)


In [3]:
# @title 未登録分（スプレッドシート）を取り込んで purchase_plan に統合
import gspread
from google.auth import default
import pandas as pd
from util.common import to_month_start, ensure_float_cols  # 既存ユーティリティ想定

# --- 認証＆シート取得 ---
creds, _ = default()
gc = gspread.authorize(creds)
SS_TITLE = "仕入予定(未登録分)"   # タイトルで開く（URLがあるなら open_by_url でもOK）
ws = gc.open(SS_TITLE).sheet1

# --- 全行→DF 化（3行目以降を使用）---
rows = ws.get_all_values()
if not rows:
    raise ValueError("シートが空です。")
df_raw = pd.DataFrame(rows)

df = df_raw.iloc[2:].copy().reset_index(drop=True)  # 3行目以降
needed_idx = [0, 6, 10, 18, 19, 20]                 # A,G,K,S,T,U
if df.shape[1] <= max(needed_idx):
    raise ValueError(f"シートの列数が不足（少なくとも {max(needed_idx)+1} 列必要）")

df = df.rename(columns={
    0:  "system_flag",   # A: '済'で除外
    6:  "sku_code",      # G
    10: "納品日",        # K
    18: "online_qty",    # S
    19: "retail_qty",    # T
    20: "wholesale_qty", # U
})

# --- '済' 以外のみ ---
df = df[df["system_flag"].astype(str).str.strip() != "済"]

# --- 納品日 → 月初 ---
dt = pd.to_datetime(df["納品日"], errors="coerce", format="%Y/%m/%d")
dt = dt.fillna(pd.to_datetime(df["納品日"], errors="coerce"))  # 形式違いの保険
df["date"] = dt.dt.to_period("M").dt.to_timestamp()            # ← datetime のまま

# --- 縦持ちへ ---
long_df = pd.melt(
    df,
    id_vars=["sku_code", "date"],
    value_vars=["online_qty", "retail_qty", "wholesale_qty"],
    var_name="channel_raw",
    value_name="qty"
)
channel_map = {"online_qty":"online", "retail_qty":"retail", "wholesale_qty":"wholesale"}
long_df["channel"] = long_df["channel_raw"].map(channel_map)
long_df["qty"] = pd.to_numeric(long_df["qty"].replace("", None), errors="coerce").fillna(0)
long_df["sku_code"] = long_df["sku_code"].astype(str).str.strip()
long_df = long_df.dropna(subset=["date", "sku_code", "channel"])

# --- 月次集計（sku_code × date × channel）---
agg = (long_df
       .groupby(["sku_code", "date", "channel"], as_index=False)["qty"]
       .sum())

# ===== ここがポイント：未登録分は「仕入_計画＝qty」「仕入_実績＝0」「仕入_未発注＝qty」として扱う =====
# ただし build_forecast_v2 は purchase_plan の形（purchase_plan_qty）を受け取るので、
# まずは purchase_plan に足し込む（未登録分すべてを “計画” として追加）:
extra_pp = agg.rename(columns={"date":"month", "qty":"purchase_plan_qty"}).copy()
extra_pp["month"] = to_month_start(extra_pp["month"])  # 月初に正規化
extra_pp = ensure_float_cols(extra_pp, ["purchase_plan_qty"])
extra_pp = extra_pp[["sku_code","channel","month","purchase_plan_qty"]]

# --- 既存 purchase_plan と統合（同月は合算）---
pp0 = frames["purchase_plan"].copy()
if "month" in pp0.columns:
    pp0["month"] = to_month_start(pp0["month"])
elif "date" in pp0.columns:
    pp0["month"] = to_month_start(pp0["date"])
pp0 = ensure_float_cols(pp0, ["purchase_plan_qty"])
pp0 = pp0[["sku_code","channel","month","purchase_plan_qty"]]

purchase_plan_merged = (pd.concat([pp0, extra_pp], ignore_index=True)
                        .groupby(["sku_code","channel","month"], as_index=False)["purchase_plan_qty"]
                        .sum())

frames["purchase_plan"] = purchase_plan_merged
print("purchase_plan merged:", frames["purchase_plan"].shape)
#display(frames["purchase_plan"].head())

purchase_plan merged: (2214, 4)


In [4]:
# @title
from util.forecasting import build_forecast_v2
from util.psi import build_psi_v2

# フォーキャスト（売上・仕入の月次テーブル）
forecast = build_forecast_v2(
    df_sales=frames["sales"],
    df_revenue=frames["revenue"],
    df_purchase=frames["purchase"],
    df_purchase_plan=frames["purchase_plan"],  # ← 未登録分マージ
)
#print("forecast:", forecast.shape)
#display(forecast.head())

# PSI（在庫推移 + 倉庫間移動）
psi = build_psi_v2(
    forecast_df=forecast,
    invent_eom_df=frames["invent_eom"],
    transfer_df=frames.get("transfer"),
)
print("PSI:", psi.shape)
display(psi.head())

PSI: (53208, 16)


,sku,channel,month,前月残,仕入_実績,仕入_計画,仕入_未発注,仕入_合計,売上_計画_フル,売上_実績,売上_計画_残,売上_合計,増減,当月残,倉庫間移動,倉庫間移動加味当月在庫
0,100000000000,online,2025-09-01,54.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,54.00000,0.0,54.00000
1,100000000000,online,2025-10-01,54.00000,3.0,0.0,0.0,3.0,13.0,15.0,0.83871,15.83871,12.83871,41.16129,0.0,41.16129
2,100000000000,online,2025-11-01,41.16129,0.0,0.0,0.0,0.0,11.0,0.0,11.00000,11.00000,11.00000,30.16129,0.0,30.16129
3,100000000000,online,2025-12-01,30.16129,0.0,0.0,0.0,0.0,9.0,0.0,9.00000,9.00000,9.00000,21.16129,0.0,21.16129
4,100000000000,online,2026-01-01,21.16129,0.0,0.0,0.0,0.0,19.0,0.0,19.00000,19.00000,19.00000,2.16129,0.0,2.16129


In [5]:
from datetime import datetime, timezone, timedelta


# 日本時間（UTC+9）で現在時刻を取得
jst = timezone(timedelta(hours=9))
ts = datetime.now(jst).strftime("%Y-%m-%d %H:%M")

psi_path      = Path("output") / f"psi_{ts}.csv"

psi.to_csv(
    psi_path,
    index=False,
    encoding="utf-8-sig",
)

print("csv保存完了:", psi_path)

csv保存完了: output/psi_2025-10-30 10:18.csv


In [6]:
from datetime import datetime, timezone, timedelta
from pathlib import Path

# ディレクトリ定義＆作成
output_dir = Path("output_tableau")
output_dir.mkdir(exist_ok=True)

# 日本時間（UTC+9）で現在時刻を取得
jst = timezone(timedelta(hours=9))
created_at = datetime.now(jst).strftime("%Y-%m-%d %H:%M")

# データフレームに作成日時カラムを追加
psi = psi.copy()
psi["作成日時"] = created_at

# ファイルパス固定
psi_path = output_dir / "psi_latest.csv"

# CSV上書き保存
psi.to_csv(
    psi_path,
    index=False,
    encoding="utf-8-sig",
)

print("latest_csv更新完了:", psi_path)


latest_csv更新完了: output_tableau/psi_latest.csv
